Link to another tutorial site that I was using to figure this out:
https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing #Convert categorical data to neumeric values

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
keep_columns=['lat','month','contest-tmp2m-14d__tmp2m',
              'climateregions__climateregion','lon']
X_columns=[['lat','month',
              'climateregions__climateregion','lon']]

In [3]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [4]:
def eliminate_columns(df):
    for col in df.columns:
        if col in keep_columns:
            print(col)
        else: 
            df=df.drop(col, axis=1)
    return df

In [5]:
#need to come up with a different way to do this, because it is not working
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [6]:
def normanlize_data(df,train_df):
    for column in df.columns:
        df[column] = df[column]  /  train_df[column].abs().max()
    return df

In [7]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [8]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [9]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [10]:
print('training set')
train_df=seperate_monthly(train_df)
train_df=eliminate_columns(train_df)
print('test set')
test_df=seperate_monthly(test_df)
test_df=eliminate_columns(test_df)

training set
lat
lon
contest-tmp2m-14d__tmp2m
climateregions__climateregion
month
test set
lat
lon
climateregions__climateregion
month


In [11]:
train_df

,lat,lon,contest-tmp2m-14d__tmp2m,climateregions__climateregion,month
139522,0.500000,0.400000,13.670195,9,5
276657,0.772727,0.900000,21.962715,8,8
224174,0.681818,0.366667,-3.850990,9,1
249296,0.727273,0.600000,11.671896,1,9
181759,0.590909,0.366667,-3.398897,1,12
...,...,...,...,...,...
304597,0.863636,0.333333,-1.765659,9,1
347177,0.954545,0.400000,19.261928,1,7
326739,0.909091,0.400000,17.471215,1,8
171878,0.545455,0.933333,4.605004,4,12


In [12]:
test_df

,lat,lon,climateregions__climateregion,month
0,0.0,0.833333,0,11
1,0.0,0.833333,0,11
2,0.0,0.833333,0,11
3,0.0,0.833333,0,11
4,0.0,0.833333,0,11
...,...,...,...,...
31349,1.0,0.866667,9,12
31350,1.0,0.866667,9,12
31351,1.0,0.866667,9,12
31352,1.0,0.866667,9,12


In [13]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [14]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [15]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [16]:
test_df

,lat,lon,climateregions__climateregion,month
0,0.0,0.833333,0.000000,0.916667
1,0.0,0.833333,0.000000,0.916667
2,0.0,0.833333,0.000000,0.916667
3,0.0,0.833333,0.000000,0.916667
4,0.0,0.833333,0.000000,0.916667
...,...,...,...,...
31349,1.0,0.866667,0.642857,1.000000
31350,1.0,0.866667,0.642857,1.000000
31351,1.0,0.866667,0.642857,1.000000
31352,1.0,0.866667,0.642857,1.000000


# Model

In [17]:
NN_model = Sequential()

#I might need to change the input_dim to give the actual dimentions
#Instead of being just 1
# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = 4, activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               640       
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 165,505
Trainable params: 165,505
Non-trainable params: 0
_________________________________________________________________


2023-02-21 05:52:24.606174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64::/opt/conda/lib
2023-02-21 05:52:24.606265: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [18]:
checkpoint_name = 'Weights--.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [19]:
NN_model.fit(X_train_df, y_train_df, epochs=20, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/20
8808/8816 [============================>.] - ETA: 0s - loss: 2.6198 - mean_absolute_error: 2.6198
Epoch 1: val_loss improved from inf to 2.25651, saving model to Weights--.hdf5
8816/8816 [==============================] - 36s 4ms/step - loss: 2.6197 - mean_absolute_error: 2.6197 - val_loss: 2.2565 - val_mean_absolute_error: 2.2565
Epoch 2/20
8807/8816 [============================>.] - ETA: 0s - loss: 2.2656 - mean_absolute_error: 2.2656
Epoch 2: val_loss improved from 2.25651 to 2.15098, saving model to Weights--.hdf5
8816/8816 [==============================] - 36s 4ms/step - loss: 2.2656 - mean_absolute_error: 2.2656 - val_loss: 2.1510 - val_mean_absolute_error: 2.1510
Epoch 3/20
8808/8816 [============================>.] - ETA: 0s - loss: 2.1701 - mean_absolute_error: 2.1701
Epoch 3: val_loss improved from 2.15098 to 2.14915, saving model to Weights--.hdf5
8816/8816 [==============================] - 35s 4ms/step - loss: 2.1701 - mean_absolute_error: 2.1701 - val_loss: 2

In [20]:
# Load wights file of the best model :
wights_file = f'Weights--.hdf5'  # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [21]:
predictions = NN_model.predict(test_df)

980/980 [==============================] - 2s 2ms/step


# Saving Dataset in correct format

In [22]:
predictions

array([[17.657803 ],
       [17.657803 ],
       [17.657803 ],
       ...,
       [-9.617395 ],
       [-9.6173935],
       [-9.6173935]], dtype=float32)

In [23]:
final=pd.DataFrame(predictions,columns=['contest-tmp2m-14d__tmp2m'])

In [24]:
i=375734
final['index']=0
while i<407088:
    final['index'][i-375734]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,17.657803,375734
1,17.657803,375735
2,17.657803,375736
3,17.657803,375737
4,17.657803,375738
...,...,...
31349,-9.617395,407083
31350,-9.617395,407084
31351,-9.617395,407085
31352,-9.617393,407086


In [25]:
final.to_csv('/kaggle/working/wetherprediction.csv',index=False)